# Building population density suitablity layers for GRIDCERF

## 1. Downloading the data

### 1.1 Download GRIDCERF

Download the GRIDCERF package if you have not yet done so from here:  https://doi.org/10.5281/zenodo.6601789.  Please extract GRIDCERF inside the `data` directory of this repository as the paths in this notebook are set to that expectation.

GRIDCERF provides mosaicked CONUS-scale population rasters for 5-year intervals from 2020 through 2100 for three Shared Socioeconomic Pathways (SSPs):  SSP2, SSP3, and SSP5.  The following is the metadata for the source data:

- **Title**:  Data Supplement: U.S. state-level projections of the spatial distribution of population consistent with Shared Socioeconomic Pathways.
- **Description from Source**:   U.S. state-level projections of the spatial distribution of population consistent with Shared Socioeconomic Pathways.
- **Source URL**:  https://doi.org/10.5281/zenodo.3756179
- **Date Accessed**:  11/3/21
- **Citation**
> Zoraghein, H., & O'Neill, B. (2020). Data Supplement: U.S. state-level projections of the spatial distribution of population consistent with Shared Socioeconomic Pathways. (v0.1.0) [Data set]. Zenodo. https://doi.org/10.5281/zenodo.3756179


## 2. Setup environment

### 2.1 Import necessary Python packages

In [2]:
import os
import glob
import math

import rasterio
import numpy as np


## 3. Configuration

In [4]:
# get the parent directory path to where this notebook is currently stored
root_dir = os.path.dirname(os.getcwd())

# data directory in repository
data_dir = os.path.join(root_dir, "data")

# GRIDCERF data directory from downloaded archive
gridcerf_dir = os.path.join(data_dir, "gridcerf")

# GRIDCERF reference data directory
reference_dir = os.path.join(gridcerf_dir, "reference")

# GRIDCERF technology specific source data directory
source_dir = os.path.join(gridcerf_dir, "source", "technology_specific", "population")

# GRIDCERF technology_specific data directory
technology_specific_dir = os.path.join(gridcerf_dir, "technology_specific")

# GRIDCERF compiled final suitability data directory
compiled_dir = os.path.join(gridcerf_dir, "compiled")


## 4. Generate population rasters

### 4.1 Functions

In [5]:
def arr_to_ascii(arr, r_ascii, xll=-180, yll=-90, cellsize=0.25, nodata=-999):
    """
    Convert a numpy array to an ASCII raster.
    :@param arr:            2D array
    :@param r_ascii:        Full path to outfile with extension
    :@param xll:            Longitude coordinate for lower left corner
    :@param yll:            Latitude coordinate for lower left corner
    :@param cellsize:       Cell size in geographic degrees
    :@param nodata:         Value representing NODATA
    """

    # get number of rows and columns of array
    nrows = arr.shape[0]
    ncols = arr.shape[1]

    # create ASCII raster file
    with open(r_ascii, 'w') as rast:

        # write header
        rast.write('ncols {}\n'.format(ncols))
        rast.write('nrows {}\n'.format(nrows))
        rast.write('xllcorner {}\n'.format(xll))
        rast.write('yllcorner {}\n'.format(yll))
        rast.write('cellsize {}\n'.format(cellsize))
        rast.write('nodata_value {}\n'.format(nodata))

        # write array
        np.savetxt(rast, arr, fmt='%.15g')
        


def create_buffer(radius):  #radius given in #cells
    
    b = []
    for i in range(radius+1):
        
        for j in range(radius+1):
            
            if math.sqrt(i**2 + j**2) <= radius:
                
                if i==0 and j==0:
                    b.extend([(i,j)])
                elif i==0:
                    b.extend([(i,j),(i,-j)])
                elif j==0:
                    b.extend([(i,j),(-i,j)])
                else:
                    b.extend([(i,j),(-i,j),(i,-j),(-i,-j)])
                    
    return b


def geotiff_to_ascii(geotiff_path, ascii_path, nodata=-999):
    
    with rasterio.open(geotiff_path) as src:
        array = src.read(1)
        with open(ascii_path, 'w') as f:
            f.write('ncols         {}\n'.format(array.shape[1]))
            f.write('nrows         {}\n'.format(array.shape[0]))
            f.write('xllcorner     {}\n'.format(src.bounds[0]))
            f.write('yllcorner     {}\n'.format(src.bounds[1]))
            f.write('cellsize      {}\n'.format(src.res[0]))
            f.write(f'NODATA_value  {nodata}\n')
            for row in array:
                for value in row:
                    f.write(' ' + str(value))
                f.write('\n')


def check_unsuitable_cells(unsuitable_cells, unsuitable_shapes, threshold):
    while len(unsuitable_cells) > 0:
        start_cell = unsuitable_cells.pop()
        to_check = [start_cell]
        this_shape = [start_cell]

        while len(to_check) > 0:
            cell = to_check.pop()
            for i, j in [(0, -1), (0, 1), (-1, 0), (1, 0)]:
                if (cell[0]+i, cell[1]+j) in unsuitable_cells:
                    to_check.append((cell[0]+i, cell[1]+j))
                    this_shape.append((cell[0]+i, cell[1]+j))
                    unsuitable_cells.remove((cell[0]+i, cell[1]+j))

        if len(this_shape) > threshold:
            unsuitable_shapes.extend(this_shape)
            
    return unsuitable_cells, unsuitable_shapes


def main(input_raster, out_dir, scenario, year=2100, buffer_km=40):
 
    buffer_relative = create_buffer(buffer_km)        #25 mile (40 km) buffer
    buffer_relative.reverse()                   #Look at biggest buffers first

    unbuffered_file = os.path.join(out_dir, f"gridcerf_densely_populated_{scenario}_{year}.asc")
    buffered_file = os.path.join(out_dir, f"gridcerf_densely_populated_{scenario}_{year}_buff25mi.asc")
    nuclear_file = os.path.join(out_dir, f"gridcerf_densely_populated_{scenario}_{year}_nuclear.asc")

    headers = []
    in_raster_data = []
    unsuitable_cells = []
    unsuitable_shapes = []
    buffered_shapes = []

    print('Loading input raster\n')
    inf = open(input_raster,'r')
    currentline = inf.readline()

    nodata_val = '-999'

    while currentline.split()[0] != nodata_val:
        headers.append(currentline)
        currentline = inf.readline()
    while currentline:
        in_raster_data.append([int(x) for x in currentline.split()])
        currentline = inf.readline()
    inf.close()

    print( 'Producing unbuffered file\n')
    #produce unbuffered file
    f = open(unbuffered_file,'w')
    for line in headers:
        f.write(line)
        
    for rowi in range(len(in_raster_data)):
        for columni in range(len(in_raster_data[rowi])):
            cell = in_raster_data[rowi][columni]
            threshold = 772
                
            if cell > threshold:
                f.write('1 ')
                unsuitable_cells.append((rowi,columni))
                
            elif cell == -9999:
                f.write('1 ')
            else:
                f.write('0 ')
        f.write('\n')
    f.close()

    print( 'Producing buffered file\n')
    print( '...identifying unsuitable blocks')
            
    threshold = 64.75

    while len(unsuitable_cells) > 0:
        start_cell = unsuitable_cells.pop()
        to_check = [start_cell]
        this_shape = [start_cell]
        
        while len(to_check) > 0:
            cell = to_check.pop()
            if (cell[0],cell[1]-1) in unsuitable_cells:
                to_check.append((cell[0],cell[1]-1))
                this_shape.append((cell[0],cell[1]-1))
                unsuitable_cells.remove((cell[0],cell[1]-1))
            if (cell[0],cell[1]+1) in unsuitable_cells:
                to_check.append((cell[0],cell[1]+1))
                this_shape.append((cell[0],cell[1]+1))
                unsuitable_cells.remove((cell[0],cell[1]+1))
            if (cell[0]-1,cell[1]) in unsuitable_cells:
                to_check.append((cell[0]-1,cell[1]))
                this_shape.append((cell[0]-1,cell[1]))
                unsuitable_cells.remove((cell[0]-1,cell[1]))
            if (cell[0]+1,cell[1]) in unsuitable_cells:
                to_check.append((cell[0]+1,cell[1]))
                this_shape.append((cell[0]+1,cell[1]))
                unsuitable_cells.remove((cell[0]+1,cell[1]))
                
        if len(this_shape) > threshold:
            unsuitable_shapes.extend(this_shape)
            
    unsuitable_cells = this_shape = None     #free up memory

    print( '...buffering')
    buffered_shapes = set(buffered_shapes)
    
    for cell in unsuitable_shapes:
        
        for offset in buffer_relative:
            buffered_shapes.add((cell[0]+offset[0], cell[1]+offset[1]))


    print( '...printing result\n')
    with open(buffered_file,'w') as f:
        for line in headers:
            f.write(line)
        for rowi in range(len(in_raster_data)):
            for columni in range(len(in_raster_data[rowi])):
                if (rowi,columni) in buffered_shapes:
                    f.write('1 ')
                elif in_raster_data[rowi][columni] == -9999:
                    f.write('1 ')
                else:
                    f.write('0 ')
            f.write('\n')
    

    print('Producing nuclear file...')
    buffer_relative=[]
    for i in range(40+1):
        buffer_relative.append(create_buffer(i))
    cell_area = 1
        
    nrows = len(in_raster_data)
    ncolumns = len(in_raster_data[0])
    
    f = open(nuclear_file,'w')
    for line in headers:
        f.write(line)
    for rowi in range(nrows):

        for columni in range(ncolumns):
            suitable = True
            break_flag = False
            for radius in buffer_relative:
                if not break_flag:
                    totalpop = 0
                    totalarea = 0

                    for offset in radius:
                        new_row = rowi + offset[0]
                        new_column = columni + offset[1]
                        if new_row >= 0 and new_row < nrows and new_column >= 0 and new_column < ncolumns and \
                           in_raster_data[new_row][new_column] != -9999:
                            totalpop += in_raster_data[new_row][new_column]
                            totalarea += cell_area

                    if totalarea != 0:
                        density = float(totalpop)/totalarea
                        if density > 193:
                            suitable = False
                            break_flag = True
                        elif density < 30:
                            break_flag = True
                    else:
                        suitable = False
                        break_flag = True
            if suitable:
                f.write('0 ')
            else:
                f.write('1 ')
        f.write('\n')
    f.close()
    
    

### 4.2 This code generates the population files on a year by year basis

In [7]:
# target SSP
ssp = "ssp2"


In [8]:
# get a list of population files for a specific SSP for all years
raster_files = glob.glob(os.path.join(source_dir, f"gridcerf_population_{ssp}_*_1km_conus.tif"))

raster_files


['/Users/d3y010/repos/metarepos/vernon-etal_2022_scidata/data/gridcerf/source/technology_specific/population/gridcerf_population_ssp2_2050_1km_conus.tif',
 '/Users/d3y010/repos/metarepos/vernon-etal_2022_scidata/data/gridcerf/source/technology_specific/population/gridcerf_population_ssp2_2100_1km_conus.tif',
 '/Users/d3y010/repos/metarepos/vernon-etal_2022_scidata/data/gridcerf/source/technology_specific/population/gridcerf_population_ssp2_2020_1km_conus.tif',
 '/Users/d3y010/repos/metarepos/vernon-etal_2022_scidata/data/gridcerf/source/technology_specific/population/gridcerf_population_ssp2_2030_1km_conus.tif',
 '/Users/d3y010/repos/metarepos/vernon-etal_2022_scidata/data/gridcerf/source/technology_specific/population/gridcerf_population_ssp2_2040_1km_conus.tif',
 '/Users/d3y010/repos/metarepos/vernon-etal_2022_scidata/data/gridcerf/source/technology_specific/population/gridcerf_population_ssp2_2060_1km_conus.tif',
 '/Users/d3y010/repos/metarepos/vernon-etal_2022_scidata/data/gridcerf

In [9]:
# convert the files to ASC rasters
for i in raster_files:
    geotiff_to_ascii(i, f"{os.path.splitext(i)[0]}.asc")


In [10]:
# get a list of population ascii for a specific SSP for all years
ascii_files = glob.glob(os.path.join(source_dir, f"gridcerf_population_{ssp}_*_1km_conus.asc"))

ascii_files


['/Users/d3y010/repos/metarepos/vernon-etal_2022_scidata/data/gridcerf/source/technology_specific/population/gridcerf_population_ssp2_2040_1km_conus.asc',
 '/Users/d3y010/repos/metarepos/vernon-etal_2022_scidata/data/gridcerf/source/technology_specific/population/gridcerf_population_ssp2_2030_1km_conus.asc',
 '/Users/d3y010/repos/metarepos/vernon-etal_2022_scidata/data/gridcerf/source/technology_specific/population/gridcerf_population_ssp2_2020_1km_conus.asc',
 '/Users/d3y010/repos/metarepos/vernon-etal_2022_scidata/data/gridcerf/source/technology_specific/population/gridcerf_population_ssp2_2050_1km_conus.asc',
 '/Users/d3y010/repos/metarepos/vernon-etal_2022_scidata/data/gridcerf/source/technology_specific/population/gridcerf_population_ssp2_2100_1km_conus.asc',
 '/Users/d3y010/repos/metarepos/vernon-etal_2022_scidata/data/gridcerf/source/technology_specific/population/gridcerf_population_ssp2_2070_1km_conus.asc',
 '/Users/d3y010/repos/metarepos/vernon-etal_2022_scidata/data/gridcerf

In [ ]:
%%time

# get a file to process; could process all in loop
f_asc = ascii_files[0]

# strip year from file
yr = int(os.path.basename(f_asc).split("_")[3])

# process file
outcome = main(f_asc, 
                 out_dir=os.path.dirname(f_asc), 
                 scenario=ssp, 
                 year=2020)


nuclear_file, headers, nrows, ncolumns, buffer_relative, in_raster_data = outcome
